In [1]:
!pip install transformers==4.40.0 accelerate
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python #양자화 구동을 위한 Llama C++ 설치

In [2]:
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'

Fetching 6 files: 100% 6/6 [00:00<00:00, 4018.82it/s]
/content


In [3]:
from llama_cpp import Llama
from transformers import AutoTokenizer

In [4]:
import pandas as pd
import random
qa_list_new = []
df_list_new = []
columns = ['질문 개수', '제시문', '프롬프트', 'instruction', '생성 질문']

In [5]:
### 연세대 2023 인문계 학종전형 p.20
presentation1 = '''
[제시문 가] : "유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다. 유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다. 남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다. 예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다. 1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다. 반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다. 유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다."

[제시문 나] : "옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다. 이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다. 변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다. 이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다. 옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다. 사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다. 예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다. 그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다. 그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다. 중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

[제시문 다] : "사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다. 가령,‘엄마, 아빠’와 같은 말은 어린이들의 말이고,‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다. 이런‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다. 젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다. 언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다. 이러한 언어의 유행 현상은 세대 변화에 따른 것이다."
'''
###[문제 1] 제시문 (가), (나), (다)에는 유행에 대한 다양한 관점이 포함되어 있다. 그관점을 비교, 분석하시오. [50점]
###[문제 2] 제시문 (라)의 조사 결과를 설명하고, 그 결과를 바탕으로 제시문 (나)와(다)의 논지를 각각 평가하시오. [50점]

###2023 연세대 학종 면접 추천형 p.17
presentation2 = '''
[제시문 가] : "인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가? 칸다울레스 왕이 다스리는 리디아 왕국에 기게스라는 목동이 살았다. 기게스가 양을 치고 있던 어느 날 갑자기 커다란 지진이 일어났다. 지진이 일어난 자리에는 땅이 갈라져 동굴이 생겼고, 기게스는 호기심이 생겨 갈라진 동굴 안으로 들어갔다. 그는 동굴 안에서 거인의 시체가 놓여 있는 것을 발견하였다. 시체의 손가락에는 금반지가 끼워져 있었다. 기게스는 그 반지를 빼 들고 밖으로 나왔다. 그러다 우연히 자신이 끼고 있는 반지의 흠집 난 곳을 안쪽으로 돌리면 자신은 투명 인간이 되고 바깥쪽으로 돌리면 자기 모습이 다시 보인다는 사실을 알게 되었다. 이제 남들의 시선에서 벗어나 보이지 않는 힘을 갖게 된 기게스는 자연스럽게 나쁜 마음을 먹게 되었다. 가축의 상태를 왕에게 보고하는 전령으로서 궁전에 들어간 기게스는 자신의 새로운 힘인 마법 반지를 이용하여 모습을 감춘 후, 왕비를 겁탈하고 그녀를 자기 편으로 끌어들여 왕을 암살한 뒤 스스로 왕이 되었다."

[제시문 나] : "서로 잘 아는 사람들이 소규모로 모여 사는 마을 공동체에서 개인은 대체로 합리적이고 도덕적인 성향을 보인다. 그는 다른 사람들의 말에 크게 영향받기보다는, 이성과 주관에 따라 판단하고 규범에 맞게 행동한다. 혼자 있을 때도, 여럿이 있을 때와 크게 다를 바 없이 처신한다.  그런데 19세기 이후 대도시에 인구가 밀집하고 대량생산과 소비, 그리고 대중문화가 발전하면서 대중사회가 등장한다.  이제 대중 속에서 이름 없는 한 명이 된 개인은 집단적인 분위기에 복종하고 전체의 결정을 따르라는 무언의 압력에 쉽사리 굴복한다. 대중의 일원으로서 그는 익명성 아래 자신의 욕망, 정열, 관심을 분출하고 실현한다. 이때 권력을 가진 지도자의 역할은 결정적이다. 권위적 지도자는 단순하고 선동적인 말로 대중에게 행동 방향을 제시하며, 대중은 지도자의 말을 마치 절대적인 진리인 것처럼 이해한다. 대중은 직관과 감정에 따라 권위적 지도자의 말을 무비판적으로 수용한다. 무리 속의 익명적 개인들은 쉽게 흥분하고 변덕을 부리며 열정을 드러낸다. 그러한 감정 에너지는 때로 대중이 난폭하게 폭력을 행사한다든지, 용감하게 순교를 불사하는 등의 행동을 하도록 만든다."

[제시문 다] : "도시 문명의 발전은 현대 사회의 주요 특징이다. 과학과 기술의 진전을 통해 이루어진 도시화는 익명성이 전통에 따르는 도덕규범과 오랜 대면 관계를 대신하도록 만들었다. 그런데 우리는 도시인의 생존이 도시의 잔인한 익명성 탓에 소모되고 훼손된다는 말을 자주 듣는다.
개인은 작은 시골 마을에 있을 때 이미 정해진 행동규범을 따르며, 스스로 그것을 의무로 여겼다. 그러한 규범을 어길 때 마을에서 평판이 나빠지기 때문이다. 하지만 그는 누가 누군지 알 수 없는 대도시로 나오면서부터 깊은 인간관계를 쌓을 수 없게 된다는 것이다. 이런 까닭에 대중사회 속의 도시인은 마치 정체성을 상실하고 자아를 잃어버린 채로 살아간다고 비판받기까지 한다. 그러나 이처럼 도시의 익명성이 과거 마을 공동체 시절의 인간적 교류를 사라지게 했다는 비판은 그 익명성이 지니는 독특한 이점을 보지 못한 데서 나온다. 도시의 익명성은 마을 공동체의 넌더리 나는 속박에서 벗어나는 자유의 가능성을 제공하므로 위협적이고 피폐한 것이 아니라 훨씬 더 인간적이고 해방적인 현상이다. 왜냐하면 도시 생활의 익명성 형태는 인간 삶에 필수적인 사생활을 보호하는 데 도움을 주며 도시인을 마을 생활의 부담스러운 도덕규범과 강요된 인습이라는 족쇄에서 벗어나도록 만들기 때문이다. 따라서 도시인은 이런 익명성 덕택에 과거와는 비교할 수 없을 만큼 많은 사람과 다양한 교류를 할 수 있게 되었을 뿐만 아니라 자유롭고 창의적인 생각을 펼칠 수 있게 되었다."
'''
###제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와(나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오. (50점)
###[문제 2] 제시문 (라)의 조사 결과를 요약하고, 제시문 (나)와 (다)를 바탕으로 그 결과를 설명하시오. (50점)

###2024 서울 p.5
presentation3 = '''
[제시문 가]"사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에 있을 과제 발표를 준비하는 데에 있어 같은 반 친구 유진이가 당신에게 도움을 줄 것인가? 당신이 유진이가 과제를 도와줄 것이라는 사실을 믿는다면, 그것은 주변 친구들을 기꺼이 도와주었던 유진이의 평소 행동 등 증거에 바탕을 둔 것일 수 있다. 반면 과제를 도와줄 것이란 사실과 관련하여 당신이 유진이라는 사람을 믿는 것은 그와의 개인적 관계에 기반한다. 설령 유진이와 친하지 않더라도 당신은 유진이가 과제를 도와줄 것이란 사실을 믿을 수 있지만, 당신이 유진이를 믿는 것은 그에 대한 당신의 개인적 태도 없이는 성립할 수 없다. ‘믿음’을 사실에 대한 믿음에, ‘신뢰’를 사람에 대한 믿음에 한정해서 말한다면, 당신이 유진이가 과제를 도와줄 것이라고 ‘믿는’ 것과 유진이가 과제를 도와줄 것이라고 ‘신뢰하는’ 것은 같은 것이 아니다."

[제시문 나]"한편, 믿음의 기반과 신뢰의 기반의 차이는 믿었던 바가 참이 아닌 것으로 드러난 경우와 신뢰했던 바가 참이 아닌 것으로 드러난 경우에 나타나는 반응의 차이를 만든다. 오후 날씨가 맑을 것이라고 믿었지만 그렇지 않은 것으로 드러났다면, 실망스럽거나 짜증이 날 수 있다. 그리고 앞으로 날씨 예측과 관련해 더 많은 증거를 찾거나 다른 종류의 증거를 찾기도 할것이다. 반면, 절도 혐의를 받고 있는 친구가 결백을 호소하여 그가 결백하다고 신뢰했지만 그렇지 않은 것으로 드러난 경우, 우리가 느끼는 바는 단지 실망스러움이나 짜증이기보다는 배신감이다."

[제시문 다]"다른 관점에서 이야기하자면, 여행을 하다가 낯선 도시에 들러 식당을 찾아갈 때, 우리는 처음 보는 사람에게 길을 묻고 그가 일러 주는 방향으로 간다. 이때 우리는 그 사람이 어떤 사람인지 특별히 아는 바가 없고, 그가 잘 알지 못하면서 무책임하게 답했다거나 우리를 골탕 먹이기 위해 엉뚱한 방향을 알려주지 않았다는 사실을 아는 것도 아니다. 그럼에도 불구하고 그 낯선 이가 말해준 방향대로 길을 간다."

[제시문 라]"또한 다른 화자는 이렇게 말한다. 사람들이 서로를 잘 신뢰하는 사회에서는 타인의 말을 쉽게 믿어 버리고 타인의 말에 더 쉽게 속을 수도 있기 때문에 거짓이 팽배해질 수 있을 것이라 생각할 수 있다. 그러나 이 사회는 거짓이 배제되고 참이 증진되는 건강한 사회로 유지된다."

'''
###[문제 1] (가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.
###[문제 2] (가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.

###2023 서울대 p.1
presentation4 = '''
[제시문 가] : "생태계가 어떻게 작동하는지 알면 알수록 많은 환경정책이 부적절하다는 사실이 드러난다. 얼핏 봐서는 상관없어 보이지만 실제로는 다른 동식물에게 유난히 큰 영향력을 미치는 종에 대해 조사하는 과정에서, 나는 친환경을 표방하는 많은 농장과 그곳의 관리체계가 빈껍데기에 불과하다는 것을 점점 더 깨닫게 되었다. 그들 농장은 많은 생물의 서식처인 나무와 관목과 죽은 나무를 잃음으로써, 물리적 구조뿐 아니라 생태계를 구성하는 다양한 종들의 관계 또한 상실했다. 그런 공간에는 생명의 거미줄이 거의 몇 줄 남아 있지 않다.

[제시문 나] : "환경 파괴와 기후 위기에 대한 경각심이 커지면서 플라스틱 빨대는 일회용품 중에서 대표적인 퇴출 대상으로 지목됐다. 하지만 플라스틱 빨대를 금지하는 정책은 빨대를 반드시 필요로 하는 사람들의 요구와 충돌한다. 빨대의 기본 형태는 오래전부터 있었지만 입구 부분이 휘어지는 플라스틱 주름 빨대는 환자들을 돕기 위해 처음 발명되었다. 플라스틱을 대체하는 친환경 빨대로 제공되는 종이 빨대, 쌀 빨대, 옥수수 전분 빨대 같은 것들은 플라스틱처럼 부드럽게 휘어지지 않아 불편하고, 뜨거운 음료에서는 쉽게 분해되므로 사용이 쉽지 않다. 플라스틱 주름 빨대를 굽은 금속 빨대 등으로 대체하는 것 역시 신체 기능이 저하된 사람들에게는 위험한 상황을 만들 수 있다. 따라서 주름 빨대를 비롯해 현대에 대량 생산되는 빨대는 부드럽고 얇은 플라스틱으로 제조되므로, 신체를 움직이기 어려운 사람들이 다른 사람의 도움 없이 음료를 마실 수 있는 유일한 방법이다.

[제시문 다] : "너희 인간들은 코로나 때문에 한 명만 죽어도 호들갑을 떨면서, 우리 동물은 수천만 마리 땅에 묻고 손을 탁탁 털더라! 자기 새끼는 끔찍이 아끼면서 남의 새끼는 끔찍하게 죽이더라! 우리의 모성애를 무시하는 당신들은 그 고매한 자식 사랑으로 무얼 했는가. 미래의 하늘에 탄소를 뿜고 미래의 땅에 분뇨 폐수 살처분 시체를 버리고 미래의 숲을 마구 베고 미래의 바다를 플라스틱으로 채운 것 말고?"
'''

###[문제 1] 환경정책을 수립할 때 유념해야 할 점에 대한 (가)와 (나)의 입장을 비교하시오.
###[문제 2] (다)의 화자를 만났을 때, (가)와 (나)의 글쓴이가 자신의 입장을 각각 어떻게 변호할지 논하시오.


new_presentation = [presentation1, presentation2, presentation3, presentation4]

In [6]:
PROMPT = '''
당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주 업무는 제시문으로부터 새로운 질문을 생성하는 것입니다.
당신은 새로운 제시문이 주어지면, 제시문들의 내용을 파악한 후 이와 관련된 질문과 답변을 생성합니다.
또한 질문들은 각 제시문 간의 관계성을 고려해 서로 비교하거나, 연관성을 물어봐야 합니다.
제시문의 전반적인 내용에 대해 골고루 질문을 생성하세요. 질문들은 특정 내용만 다루는 것이 아닌, 제시문 전체를 구체적으로 다뤄야 합니다.

예를 들어 다음과 같은 제시문 3개가 있습니다.

###
[가]진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의 경계에 위치하면서 다른 학문보다 더 어렵기도 하고 더 쉽기도 하다. 역사가들은 원하는 정보 모두를 획득할 때까지 사료를 끊임없이 파헤치고, '사실'을 다루는 자신들의 깊이를 앞세워 여타 학문의 동료들을 괴롭히는 콧대 높은 경험주의자들이다. 이와 동시에 역사책은 흔히 이야기를 중심으로 전개되며, 가장 성공적인 역사서들은 대체로 훌륭한 소설의 속성을 일정하게 갖고 있다.
역사학의 본질적 혼종성은 과거를 재구성하는 데 있어서 사실성과 허구성 사이의 경계에 관한 논쟁의 핵심적 이유이다.

[나]크리스토퍼 브라우닝(Christopher Browning)은 1942~1943년에 걸쳐 약 38,000명의 유대인 학살 명령을 수행한 독일 101예비경찰대의 재판 기록을 통해 ‘평범한 사람들’이 학살에 가담했던 이유를 설명한다. 유대인을 죽이라는 명령을 받고 당황한 대원들에게 상관은 나이가 좀 더 많은 사람들은 임무를 수행하지 못할 것 같으면 빠져도 좋다고 말했지만, 선택의 가능성에도 불구하고 80~90%의 대원들이 대량 학살에 가담했다. 브라우닝은 사회적 관계로 인해 나약한 인간이 부당한 일을 행할 수 있다고 보았다. 순응주의, 권위에 대한 복종, 임무를 거부할 때 동료들로부터 따돌림을 당할지도 모른다는 두려움이 학살 가담의 결정적 원인이라는 것이다. 브라우닝은 무엇이 보통 사람들을 그토록 잔혹한 범죄에 가담하도록 이끌었는가를 이해하려 했던 것이고 그의 결론은 집단적 순응성의 압도적인 영향이었다.

[다]대니얼 골드하겐(Daniel Goldhagen)은 브라우닝과 동일한 사료를 검토하고 정반대 결론을 내렸다. 그의 결론은 101예비경찰대의 압도적 다수가 동료들의 압력, 복종, 혹은 자신들의 경력 때문에 학살에 가담했던 것이 아니라, 섬뜩할 정도로 냉담하고 잔인한 행동을 묘사한 기록들에서 드러나듯 유대인 학살의 적극적 욕망을 가지고 행동했기 때문이라는 것이다. 골드하겐은, 학살 가담이 내키지 않았고 자신들의 행동을 혐오했다는 대원들의 진술이 자기 변호에 불과하며, 그들은 ‘평범한 보통 사람들’이 아니라 ‘비정상적인 정치문화의 보통 사람들’이라고 보았다. 그의 명제는 단순하고 명확하다. “독일인의 반유대주의적 신념이 홀로코스트를 유발한 핵심 동인이다.” 골드하겐은 사회적 관계에 초점을 맞추기보다는 반유대주의라는 당시 독일 사회의 특수성을 문제시했다. 그의 자명한 주장은 앞선 역사가들과 달랐지만, 상당한 대중적 찬사를 받았다.
###

이 제시문 세트에서 출제된 문제는 다음과 같습니다.
###
1. (가)에서 말한 역사학에서의 허구성을 구체적으로 설명하고, (나)와 (다)에서 발견되는 허구적 요소가 각각 무엇인지 설명하시오.
2. (가)에서 말한 ‘혼종성’이 다른 학문 분야에서 어떻게 나타날 수 있는지 예를 들어 설명하시오.
###

이처럼 새로운 제시문이 주어지면 이에 대한 질문을 생성하면 됩니다.
이때 생성할 질문들은 예시 질문들과 유사한 형태이어야 합니다.
다음은 예시 질문들입니다.

["(나), (다), (라) 각각에 근거하여 (가)에 나타난 정부의 물가 변동 파악 방식의 한계점을 설명하시오."]
["(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오."]
["(가)와 (나)에 나타난 독자에 대한 공통된 이해 방식을 (다)의 맥락에서 평가하시오."]
["(가)와 (나)의 밑줄 친 사례에서 관찰되는 문제점의 공통점과 차이점을 구체적으로 설명하시오.]
["독자와 저자(혹은 작가)의 관계에 관해 (가)와 (나)에 제시된 입장을 비교하시오."]
["(다)의 화자를 만났을 때, (가)와 (나)의 글쓴이가 자신의 입장을 각각 어떻게 변호할지 논하시오."]

이처럼 모든 질문은 반드시 "~하시오."로 끝나야 합니다.
'''

In [7]:
presentation_number = random.randint(0,2)
num_of_question = random.randint(3,5)

!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'


model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)

model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

generation_kwargs = {
    "max_tokens":2000,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.3,
    "echo":True, # Echo the prompt in the output
}
df_list_new = []
qa_list_new = []
for i in range(4) :
  num_of_question = random.randint(3,5)
  if num_of_question == 3 :
    format = '''
    "질문 1"
    "답변 1"
    "질문 2"
    "답변 2"
    "질문 3"
    "답변 3"'''
  elif num_of_question == 4 :
    format = '''
    "질문 1"
    "답변 1"
    "질문 2"
    "답변 2"
    "질문 3"
    "답변 3"
    "질문 4"
    "답변 4"'''

  else :
    format = '''
    "질문 1"
    "답변 1"
    "질문 2"
    "답변 2"
    "질문 3"
    "답변 3"
    "질문 4"
    "답변 4"
    "질문 5"
    "답변 5"'''

  test_text = f'''다음 제시문을 보고, 질문과 답변 세트 {num_of_question}개 생성하라.

  새로운 제시문은 다음과 같다.
  ###
  {new_presentation[i]}
  ###

  생성 형식은
  {format}
  으로 해야한다.

  주의점은 다음과 같다.
  ###
  1. 다른 불필요한 문장은 생성하지 마시오. 오직 질문과 답변만 생성하시오.
  2. 질문은 반드시 한국어로, 1~2문장으로 구성해야 한다.
  3. 각 질문들은 서로 다른 형태와 내용으로, 겹치지 않아야 한다.
  4. 모든 질문은 반드시 "~하시오."의 명령조 형식으로 끝나야 한다.
  5. 또한 질문은 최대한 자연스러운 말투이어야 한다.
  6. 마지막으로 질문에는, 답변에 대한 내용이 나타나서는 안 된다.
  ###

  예를 들어, 질문은
  ###
  "제시문 [가]와 [나]에 기반하여 [다]에 대한 ~~~를 설명하시오."
  "제시문 [가]를 기반으로 [나]의 ~~~~에 대해 논의하시오."
  ###
  등의 형식이어야 한다.
                  '''

  messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

  prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0)

  resonse_msg = model(prompt, **generation_kwargs)
  generated = resonse_msg['choices'][0]['text'][len(prompt):]
  print(generated)
  print(f'질문 개수 : {num_of_question}')
  qa_list_new.append(generated)
  df_list_new.append([num_of_question, new_presentation[presentation_number], PROMPT, test_text, generated])
  print('\n')

Fetching 6 files: 100% 6/6 [00:00<00:00, 61230.72it/s]
/content


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


이 문제를 해결하기 위해 생성된 질문 및 답변 세트는 다음과 같습니다.
  
 질문 1:
 제시문 [가]와 [나]의 비교를 통해, 유행이 어떤 사회적 현상을 나타내는지 설명하십시오.

답변 1:
 제시문 [가]와 [나]는 모두 유행과 관련된 내용을 담고 있습니다. 제시문 [가]는 유행이 사람들이 어떤 행동을 하는지에 영향을 미치는 것으로 설명하며, 이는 사람들이 주변의 다른 사람들과 일치하기를 원하기 때문입니다. 반면에 제시문 [나]는 유행이 사람들 사이의 동질감을 형성하는 역할을 한다는 것을 강조합니다. 사람들은 새로운 것을 따라하고, 변화를 수용함으로써 주변 사람들과 함께 되기를 추구한다는 것입니다.
  
 질문 2:
 제시문 [다]를 기반으로, 언어가 어떻게 유행에 영향을 미치는지 설명하십시오.

답변 2:
 제시문 [다]는 젊은 세대가 언어를 통해 자신들의 개성과 특징을 표현하고자 한다는 것을 보여줍니다. 이들은 새로운 유행어와 신조어를 만들어내고, 이를 통해 themselves와 다른 사람들을 구분합니다. 따라서 언어는 유행을 따라가는 사람들의 자아 표현의 수단이며, 새로운 변화가 쉽게 수용되는 또 다른 특징입니다. 이 말은 사람들이 소셜미디어 및 문화산업과 같은 유행 형성 요인에 더욱 노출되어 있음을 시사합니다.
  
 질문 3:
 제시문 [가], [나], 및 [다] 사이의 관계점을 설명하십시오.

답변 3:
 제시문 [가], [나], 및 [다]는 모두 유행에 대한 다양한 측면을 다루고 있습니다. 제시문 [가]는 유행이 행동을 조정하는 것을 강조하며, 제시문 [나]는 동질감을 형성하는 역할을 나타냅니다. 마지막으로 제시문 [다]는 언어와 같은 사회적인 요소가 유행에 영향을 미치는 것을 보여줍니다. 따라서 이 세 가지 진술은 모두 서로 다른 관점을 통해 유행을 이해하는 데 도움이 됩니다. 상호 비교와 연관성을 고려하면, 우리는 다양한 시각에서 유행이 어떻게 사회적 현상을 형성하고 우리의 행동을 어떤 방식으로 조정하는지 더 잘 이해할 수 있습니다.
질문

In [8]:
df_new = pd.DataFrame(df_list_new, columns=columns)
df_new

,질문 개수,제시문,프롬프트,instruction,생성 질문
0,3,"\n[제시문 가]""사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라.\n\n 새로운 제시문은 다...",이 문제를 해결하기 위해 생성된 질문 및 답변 세트는 다음과 같습니다.\n \n ...
1,5,"\n[제시문 가]""사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 5개 생성하라.\n\n 새로운 제시문은 다...",질문 1:\n제시문 가]와 제시문 나]를 비교하여 공통점과 차이점을 설명하시오.\n...
2,3,"\n[제시문 가]""사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라.\n\n 새로운 제시문은 다...",이 문제는 제시문을 바탕으로 질문과 답변을 세 세트로 생성하는 것입니다. 각 섹션에...
3,4,"\n[제시문 가]""사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 4개 생성하라.\n\n 새로운 제시문은 다...",질문 1:\n제시문 [가]와 [나]를 비교하여 공통점과 차이점을 설명하시오.\n\n...


In [9]:
df_new.to_csv('0626 시도_14시.csv', encoding = 'utf-8')